In [1]:
import sys
import re
sys.path.append('/Users/yuhangzhan/Git/statement_reader')

In [2]:
from src.model.bank_statement import BankStatement
from statement.boa import BankOfAmerica_Credit
from statement.chase import Chase_Credit

In [3]:
boa_fpath = '/Users/yuhangzhan/Git/statement_reader/data/pdf/boa/credit/eStmt_2018-01-28.pdf'
chase_fpath = '/Users/yuhangzhan/Git/statement_reader/data/pdf/chase/credit/20190504-statements-9090-.pdf'
boa = BankOfAmerica_Credit(boa_fpath)
chase = Chase_Credit(chase_fpath)

In [ ]:
isinstance(boa, BankStatement)

In [ ]:
boa.summary

In [5]:
boa.transactions

{'payment': [{'types': 'payment',
   'transaction_date': '2018-01-04',
   'posting_date': '2018-01-04',
   'reference_number': 1352,
   'account_number': 7577,
   'amount': -300.0,
   'description': 'Online  from CHK 8'},
  {'types': 'payment',
   'transaction_date': '2018-01-19',
   'posting_date': '2018-01-19',
   'reference_number': 5258,
   'account_number': 7577,
   'amount': -1000.0,
   'description': 'Online  from CHK 8'},
  {'types': 'payment',
   'transaction_date': '2018-01-27',
   'posting_date': '2018-01-27',
   'reference_number': 1786,
   'account_number': 1528,
   'amount': -26.34,
   'description': 'EB SMSA R WORKSHOP FO       801-413-7200 CA'},
  {'types': 'payment',
   'transaction_date': '2018-01-27',
   'posting_date': '2018-01-27',
   'reference_number': 4016,
   'account_number': 7577,
   'amount': -31.72,
   'description': 'Amazon.com          AMZN.COM/BILLWA'},
  {'types': 'payment',
   'transaction_date': '2018-01-27',
   'posting_date': '2018-01-27',
   'refer

In [ ]:
chase.raw_pdf_string.split('Totals Year-to-Date')[1]

In [ ]:
re.findall(r"Purchases\s+([-|+])?\$([\d|,]+\.\d+)\s", chase.raw_pdf_string)

In [ ]:
chase.summary

In [4]:
chase.transactions

{'fee': [{'type': 'fee',
   'transaction_date': '05/01/2019',
   'posting_date': 'Null',
   'ref_number': 'Null',
   'account_number': 9090,
   'amount': 450.0,
   'description': 'ANNUAL MEMBERSHIP FEE'}],
 'purchase': [{'type': 'purchase',
   'transaction_date': '04/20/2019',
   'posting_date': 'Null',
   'ref_number': 'Null',
   'account_number': 9090,
   'amount': 9.77,
   'description': '99 RANCH #1621 GAITHERSBURG MD'},
  {'type': 'purchase',
   'transaction_date': '04/19/2019',
   'posting_date': 'Null',
   'ref_number': 'Null',
   'account_number': 9090,
   'amount': 49.0,
   'description': 'COURSRA*7YAYWCGJQTPT2V 650-265-7649 CA'},
  {'type': 'purchase',
   'transaction_date': '04/18/2019',
   'posting_date': 'Null',
   'ref_number': 'Null',
   'account_number': 9090,
   'amount': 9.53,
   'description': 'BIBIBOP ASIAN GRILL - BET BETHESDA MD'},
  {'type': 'purchase',
   'transaction_date': '04/19/2019',
   'posting_date': 'Null',
   'ref_number': 'Null',
   'account_number': 9